# Experiment 2
*Question*: how does generalization error (error on validation set) change as we reduce the amount of unique training examples for both the subspace-constrained and non-subspace-constrained methods?
*Hypothesis*: Subspace constrained method will have less generalization error than non subspace constrained method as the number of unique training examples decreases
*Todo*: use subspace dimension from experiment 1

In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.cnn import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from cs231n.layers import *
from cs231n.fast_layers import *
from cs231n.solver import Solver
from copy import deepcopy

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

def rel_error(x, y):
  # returns relative error
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Load the preprocessed CIFAR10 data.
alldata = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

X_train:  (49000, 3, 32, 32)
y_train:  (49000,)
X_val:  (1000, 3, 32, 32)
y_val:  (1000,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


In [ ]:
ntrain_total = data['X_train'].shape[0]
pct_train_sweep = (1, 0.8, 0.6, 0.4, 0.2)
results_train_accuracy = np.zeros((len(pct_train_sweep),2))
results_test_accuracy  = np.zeros((len(pct_train_sweep),2))
for (i,pct_train) in enumerate(pct_train_sweep):
    # -------------------------
    # --- generate data set ---
    # -------------------------
    ntrain_unique = round(pct_train*ntrain_total)
    ntrain_dupl = ntrain_total - ntrain_unique
    ind_unique = np.arange(0,ntrain_unique)
    ind_dupl = np.random.choice(np.arange(0,ntrain_unique),size=)[0:ntrain_dupl]
    ind_train = np.concatenate((ind_unique, ind_dupl))
    print('----- trial %d: %d percent of data (%d unique examples, %d duplicate examples) -----' % (i, pct_train*100, ntrain_unique, ntrain_dupl))
    data_abbrev['X_train'] = deepcopy(alldata['X_train'])[ind_train,:,:,:]
    data_abbrev['y_train'] = deepcopy(alldata['y_train'])[ind_train]
    data_abbrev['X_val']   = deepcopy(alldata['X_val'])
    data_abbrev['y_val']   = deepcopy(alldata['y_val'])
    data_abbrev['X_test']  = deepcopy(alldata['X_test'])
    data_abbrev['y_test']  = deepcopy(alldata['y_test'])
    # --------------------------------------
    # --- train and report test accuracy ---
    # --------------------------------------
    standardModel  = ThreeLayerConvNet(weight_scale=0.001, hidden_dim=600, reg=0.001)
    subspaceModel  = ThreeLayerConvNet(weight_scale=0.001, hidden_dim=600, reg=0.001)
    standardSolver = Solver(standardModel, data_abbrev,
                            num_epochs=2, batch_size=50,
                            update_rule='adam',
                            optim_config={
                              'learning_rate': 1e-4,
                            },
                            verbose=True, print_every=100)
    subspaceSolver = Solver(subspaceModel, data_abbrev,
                            num_epochs=2, batch_size=50,
                            update_rule='adam',
                            optim_config={
                                'learning_rate': 1e-4,
                            },
                            verbose=True, print_every=100)
    ambient_dim = data_abbrev['X_train'].shape[2]*data_abbrev['X_train'].shape[3]
    print('=== TRAINING STANDARD MODEL FOR TRIAL %d ===' % i)
    standardSolver.train(ambient_dim)
    results_train_accuracy[i,0] = solver.train_acc_history[-1]
    results_test_accuracy[i,0] = solver.test_acc_history[-1]
    print('final accuracy for %d percent of data, standard model: %.4f (train), %.4f (test)' % (pct_train*100, results_train_accuracy[i,0], results_test_accuracy[i,1]))
    print('=== TRAINING SUBSPACE MODEL FOR TRIAL %d ===' % i)
    subspaceSolver.train(round(0.5*ambient_dim))
    results_train_accuracy[i,1] = subspaceSolver.train_acc_history[-1]
    results_test_accuracy[i,1] = subspaceSolver.test_acc_history[-1]
    print('final accuracy for %d percent of data, subspace model: %.4f (train), %.4f (test)' % (pct_train*100, results_train_accuracy[i,0], results_test_accuracy[i,1]))

----- trial 0: 100 percent of data (49000 unique examples, 0 duplicate examples) -----
=== TRAINING STANDARD MODEL FOR TRIAL 0 ===
(Iteration 1 / 1960) loss: 2.305161
(Epoch 0 / 2) train acc: 0.104000; val_acc: 0.112000
(Iteration 101 / 1960) loss: 1.579576
(Iteration 201 / 1960) loss: 1.655479
(Iteration 301 / 1960) loss: 1.463823
(Iteration 401 / 1960) loss: 1.350504
(Iteration 501 / 1960) loss: 1.333810
(Iteration 601 / 1960) loss: 1.376882
(Iteration 701 / 1960) loss: 1.440234
(Iteration 801 / 1960) loss: 1.134020
(Iteration 901 / 1960) loss: 1.106834
(Epoch 1 / 2) train acc: 0.537000; val_acc: 0.552000
(Iteration 1001 / 1960) loss: 1.711220
